Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #[5 x 5 x 1 x 16]
  layer1_biases = tf.Variable(tf.zeros([depth])) #[16]
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1)) #[5 x 5 X 16 x 16]
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.522577
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 50: 1.780909
Minibatch accuracy: 37.5%
Validation accuracy: 49.0%
Minibatch loss at step 100: 1.043371
Minibatch accuracy: 62.5%
Validation accuracy: 65.5%
Minibatch loss at step 150: 0.536843
Minibatch accuracy: 81.2%
Validation accuracy: 74.9%
Minibatch loss at step 200: 0.803869
Minibatch accuracy: 81.2%
Validation accuracy: 77.5%
Minibatch loss at step 250: 1.143996
Minibatch accuracy: 68.8%
Validation accuracy: 77.5%
Minibatch loss at step 300: 0.378415
Minibatch accuracy: 93.8%
Validation accuracy: 80.2%
Minibatch loss at step 350: 0.505164
Minibatch accuracy: 93.8%
Validation accuracy: 77.2%
Minibatch loss at step 400: 0.254245
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.950667
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 500: 0.743572
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [21]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  layer3_pool_weights = tf.Variable(tf.truncated_normal(
      [2 * 2 * depth, num_hidden], stddev=0.1))
  layer3_pool_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_pool_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_pool_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    print(hidden)
    h_pool1 = max_pooling(hidden)
    print(h_pool1)
    conv = tf.nn.conv2d(h_pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    h_pool2 = max_pooling(hidden)
    shape = h_pool2.get_shape().as_list()
    print(shape) #[16, 2, 2, 16]
    reshape = tf.reshape(h_pool2, [shape[0], shape[1] * shape[2] * shape[3]]) #[16 - 64]
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  def max_pooling(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 1001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Tensor("Relu:0", shape=(16, 28, 28, 16), dtype=float32)
Tensor("MaxPool:0", shape=(16, 14, 14, 16), dtype=float32)
[16, 7, 7, 16]
Tensor("Relu_3:0", shape=(10000, 28, 28, 16), dtype=float32)
Tensor("MaxPool_2:0", shape=(10000, 14, 14, 16), dtype=float32)
[10000, 7, 7, 16]
Tensor("Relu_6:0", shape=(10000, 28, 28, 16), dtype=float32)
Tensor("MaxPool_4:0", shape=(10000, 14, 14, 16), dtype=float32)
[10000, 7, 7, 16]
Initialized
Minibatch loss at step 0: 3.430592
Minibatch accuracy: 6.2%
Validation accuracy: 6.9%
Minibatch loss at step 50: 1.868213
Minibatch accuracy: 18.8%
Validation accuracy: 48.8%
Minibatch loss at step 100: 1.050713
Minibatch accuracy: 68.8%
Validation accuracy: 69.5%
Minibatch loss at step 150: 0.370910
Minibatch accuracy: 81.2%
Validation accuracy: 74.5%
Minibatch loss at step 200: 1.266725
Minibatch accuracy: 56.2%
Validation accuracy: 78.1%
Minibatch loss at step 250: 1.495162
Minibatch accuracy: 56.2%
Validation accuracy: 78.3%
Minibatch loss at step 300: 0.369817


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [25]:
# batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  layer3_pool_weights = tf.Variable(tf.truncated_normal(
      [2 * 2 * depth, num_hidden], stddev=0.1))
  layer3_pool_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_pool_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_pool_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))


   # dropout keep rate  
  keepRate = tf.placeholder(tf.float32, shape=())
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    h_pool1 = max_pooling(hidden)
    conv = tf.nn.conv2d(h_pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    h_pool2 = max_pooling(hidden)
    shape = h_pool2.get_shape().as_list()
    #shape [16, 2, 2, 16]
    reshape = tf.reshape(h_pool2, [shape[0], shape[1] * shape[2] * shape[3]]) #[16 - 64]
        
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    droppedOutReluResult = tf.nn.dropout(hidden,keepRate)

    
    return tf.matmul(droppedOutReluResult, layer4_weights) + layer4_biases
  
  def max_pooling(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 5001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:0.6}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:1.0}), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:1.0}), test_labels))

Initialized
Minibatch loss at step 0: 3.830875
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.291079
Minibatch accuracy: 18.8%
Validation accuracy: 14.3%


KeyboardInterrupt: 

# Lenet5

In [29]:
batch_size = 16
patch_size = 5
depth = 32
depth2 = 32
num_hidden = 512

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(0.1, shape=[depth2]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 7 * image_size // 7 * depth2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(0.1, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))


   # dropout keep rate  
  keepRate = tf.placeholder(tf.float32, shape=())
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer1_biases)
    #[24 *24]
    h_pool1 = max_pooling(hidden)
    #[12 * 12]
    conv = tf.nn.conv2d(h_pool1, layer2_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer2_biases)
    #[8*8]
    h_pool2 = max_pooling(hidden)
    #[4*4]
    shape = h_pool2.get_shape().as_list()
    #shape [16, 4, 4, 16]
    reshape = tf.reshape(h_pool2, [shape[0], shape[1] * shape[2] * shape[3]]) #[16 - 64]
    
    #layer3 weight = [4*4*depth]
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    droppedOutReluResult = tf.nn.dropout(hidden,keepRate)

    return tf.matmul(droppedOutReluResult, layer4_weights) + layer4_biases
  
  def max_pooling(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 5000, 0.80, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  

  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  #optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 5001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:0.6}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:1.0}), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:1.0}), test_labels))

Initialized
Minibatch loss at step 0: 3.517152
Minibatch accuracy: 6.2%
Validation accuracy: 12.6%
Minibatch loss at step 50: 1.419991
Minibatch accuracy: 56.2%
Validation accuracy: 69.3%
Minibatch loss at step 100: 0.675867
Minibatch accuracy: 75.0%
Validation accuracy: 75.5%
Minibatch loss at step 150: 0.596685
Minibatch accuracy: 81.2%
Validation accuracy: 74.8%
Minibatch loss at step 200: 0.966594
Minibatch accuracy: 81.2%
Validation accuracy: 78.1%
Minibatch loss at step 250: 1.038119
Minibatch accuracy: 68.8%
Validation accuracy: 79.3%
Minibatch loss at step 300: 0.478591
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
Minibatch loss at step 350: 0.621232
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 400: 0.397626
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Minibatch loss at step 450: 1.152383
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 500: 0.807769
Minibatch accuracy: 75.0%
Validation accuracy: 82.8%
Mi